** Final project code **

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import pygsp as pg

In [3]:
from load_and_preprocessing import load_data_and_filter_members, assign_party_to_names
from visualization import label_to_numeric, get_lap_eigendecomp, detect_partitions, make_signal
from conseil_national_evolution import create_evolution_features_v1

In [7]:
# Use load_data_and_filter_members to create adjacency for any file from any legislature
# Example 
leg='49'
from_date = '01_01_2012'
to_date = '31_12_2012'
adjacency, node_index, sum_na_per_row = load_data_and_filter_members('../data/abdb-de-all-affairs-'+leg+'-0.csv',
                                                                     start_date=from_date, end_date=to_date,
                                                                     filter_method='number_NA',cutoff=10,ret_transf=False)

(Nbr. of councillors, nbr. of votes) before filter: (204, 419)
(Nbr. of councillors, nbr. of votes) after filter: (194, 418)


Assigning parties to the councillors from the dataset loaded above:

In [9]:
name_with_party = assign_party_to_names('../data/Ratsmitglieder_1848_FR.csv', node_index)
print(name_with_party.to_string())

         Counc_Id                  CouncillorName PartyAbbreviation
node_id                                                            
0             490              Lustenberger Ruedi               PDC
1            4025              Büchel Roland Rino               UDC
2             504                Rossini Stéphane               PSS
3            3922                   Favre Laurent               PLR
4            4092                    Poggia Mauro               MCR
5            1104                   Büchler Jakob               PDC
6            3911                      Voruz Eric               PSS
7            1295             Graf-Litscher Edith               PSS
8            4052                   Ritter Markus               PDC
9            3876                Estermann Yvette               UDC
10           4075                   Caroni Andrea               PLR
11           1112                     Müller Geri               PES
12           3892                     Killer Han

Make a csv that can be used in Gephi:

In [10]:
name_labels = name_with_party['CouncillorName'].values
adjacency_df = pd.DataFrame(data=adjacency, columns=name_labels, index=name_labels)
adjacency_df.to_csv('gephi_'+leg+'-'+from_date+'-'+to_date+'.csv', sep=',', index_label='', index=True, header=True)

Translating the party label into a numerical value:

In [11]:
# Example here is with parties, but the same function works for any kind of dictionary, eg. lobbying mandates
party_map = {'UDC': 6,'PSS': -6,'PDC':0,'BastA':-3,'PLR':5,'pvl':-2,
             'PES':-5, 'PBD':2, 'PdT':-4,'PLS':5,
             'PRD':5, 'MCR':4, 'PEV':-1, 'Lega':3, 'csp-ow':1, 'Al':-7,'FraP!':-8, 'GB':7, 'CSPO':8 }

name_with_party_num, labels_in_data = label_to_numeric(name_with_party, 'PartyAbbreviation', party_map, ret_values=True)

In [12]:
name_labels = name_with_party['CouncillorName'].values
d= name_with_party_num['PartyAbbreviation'].values
Party_attribute = pd.DataFrame(data=np.c_[name_labels,d], columns=['Id','party'])
Party_attribute.to_csv('gephi_'+leg+'-'+from_date+'-'+to_date+'party'+'.csv', sep=',',  index=False, header=True)

This can be used to see if the label dictionary needs to be adjusted:

In [13]:
print("Unique label values found in data: \n{0}".format(labels_in_data))
print("Label values that are translated by provided dictionary: \n{0}".format(party_map.keys()))

Unique label values found in data: 
['PDC' 'UDC' 'PSS' 'PLR' 'MCR' 'PES' 'Lega' 'PBD' 'pvl' 'csp-ow' 'PEV'
 'PLS']
Label values that are translated by provided dictionary: 
dict_keys(['UDC', 'PSS', 'PDC', 'BastA', 'PLR', 'pvl', 'PES', 'PBD', 'PdT', 'PLS', 'PRD', 'MCR', 'PEV', 'Lega', 'csp-ow', 'Al', 'FraP!', 'GB', 'CSPO'])


Find partitions based on Louvain method and calculate modularity

In [14]:
partitions, modularity = detect_partitions(adjacency, resolution=1)

Draw some eigenmaps and plot party membership as signal:

In [15]:
eigenvals, eigenvectors = get_lap_eigendecomp(adjacency, lap_type='normalized', ret_eigval=True)

In [ ]:
%pylab

In [ ]:
partition_signal = make_signal(adjacency.shape[0],partitions)
party_signal = name_with_party_num['PartyAbbreviation'].values
colorbar_labels = ['UDC', 'PLR', 'MCR', 'LEGA', 'PBD', 'csp-ow', 'PDC', 'PEV', 'pvl', 'BastA', 'PdT', 'PES', 'PSS']

In [ ]:
fig = plt.figure(figsize=(16,9))
ax = fig.add_subplot(111)
ax.scatter(eigenvectors[:,1], eigenvectors[:,2], c=partition_signal, s=15, vmin=-6, vmax=6, cmap="jet")

for i, txt in enumerate(name_labels):
    ax.annotate(txt, (eigenvectors[i,1], eigenvectors[i,2]), xytext=(1,1), textcoords='offset points')
    
ax.set_title('Position on political spectrum by party ')
fig.tight_layout()

Set colorbar to whatever

In [ ]:
cbar = fig.colorbar(ax.collections[0], ticks=np.linspace(6,-6,13), orientation='vertical')
cbar.ax.set_yticklabels(colorbar_labels)

**Party orientation assessment**

In [16]:
legislatures=['48','49','50']
# Compute average orientation of a party per year
party_evolution_df, years = create_evolution_features_v1(legislatures)

Legislature: 48
Year: 1
(Nbr. of councillors, nbr. of votes) before filter: (202, 146)
(Nbr. of councillors, nbr. of votes) after filter: (196, 145)
Flip sign
Year: 2
(Nbr. of councillors, nbr. of votes) before filter: (202, 604)
(Nbr. of councillors, nbr. of votes) after filter: (196, 603)
Year: 3
(Nbr. of councillors, nbr. of votes) before filter: (207, 411)
(Nbr. of councillors, nbr. of votes) after filter: (191, 410)
Flip sign
Year: 4
(Nbr. of councillors, nbr. of votes) before filter: (203, 528)
(Nbr. of councillors, nbr. of votes) after filter: (195, 528)
Flip sign
Legislature: 49
Year: 1
(Nbr. of councillors, nbr. of votes) before filter: (204, 419)
(Nbr. of councillors, nbr. of votes) after filter: (194, 418)
Year: 2
(Nbr. of councillors, nbr. of votes) before filter: (206, 473)
(Nbr. of councillors, nbr. of votes) after filter: (192, 473)
Year: 3
(Nbr. of councillors, nbr. of votes) before filter: (209, 358)
(Nbr. of councillors, nbr. of votes) after filter: (188, 358)
Flip si

C:\Users\silus\Anaconda3\lib\site-packages\numpy\core\_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Users\silus\Anaconda3\lib\site-packages\numpy\core\_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\Users\silus\Anaconda3\lib\site-packages\numpy\core\_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [17]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [18]:
party_to_be_plotted = ['UDC','PSS','PDC','pvl','PLR','PES','PBD']
party_colors = ['royalblue','r', 'orange', 'g', 'cyan', 'forestgreen', 'yellow']
party_color_map = dict((key, value) for (key, value) in zip(party_to_be_plotted, party_colors))

fig = plt.figure(figsize=(16,9))
ax = fig.add_subplot(111)
for party in party_to_be_plotted:
    orientation = []
    domination = []
    for values_per_year in party_evolution_df:
        values_party = values_per_year[party].values
        orientation.append(values_party[0])
        domination.append(values_party[1])
    ax.plot(orientation, years, c=party_color_map[party])
plt.yticks(years, years.astype(str))

([<matplotlib.axis.YTick at 0x1b6027481d0>,
 <a list of 11 Text yticklabel objects>)